In [1]:
import sys
import cv2 as cv2
from random import randint
 
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
 
    
def createTrackerByName(trackerType):
    # Create a tracker based on tracker name
    
    if trackerType == trackerTypes[0]:
        tracker = cv2.TrackerBoosting_create()
  
    elif trackerType == trackerTypes[1]:
        tracker = cv2.TrackerMIL_create()
        
    elif trackerType == trackerTypes[2]:
        tracker = cv2.TrackerKCF_create()
        
    elif trackerType == trackerTypes[3]:
        tracker = cv2.TrackerTLD_create()
        
    elif trackerType == trackerTypes[4]:
        tracker = cv2.TrackerMedianFlow_create()
        
    elif trackerType == trackerTypes[5]:
        tracker = cv2.TrackerGOTURN_create()
        
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
        
    elif trackerType == trackerTypes[7]:
        tracker = cv2.TrackerCSRT_create()
        
    else:
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers are:')
        for t in trackerTypes:
            print(t)
        
    return tracker


In [2]:
videoPath = "surveillance.mp4"
# Create a video capture object to read videos

cap = cv2.VideoCapture(videoPath)

 
# Read first frame
success, frame = cap.read()

# quit if unable to read the video file

if not success:
    print('Failed to read video')
    sys.exit(1)


In [3]:
bboxes = []
colors = []

# OpenCV's selectROI function doesn't work for selecting multiple objects in Python
# So we will call this function in a loop till we are done selecting all objects
while True:
    
    # draw bounding boxes over objects
    # selectROI's default behaviour is to draw box starting from the center
    # when fromCenter is set to false, you can draw box starting from top left corner
    bbox = cv2.selectROI('MultiTracker', frame)
    bboxes.append(bbox)
    colors.append((randint(0, 255), randint(0, 255), randint(0, 255)))
    print("Press q to quit selecting boxes and start tracking")
    print("Press any other key to select next object")
    k = cv2.waitKey(0) & 0xFF
    
    if (k == 113):  # q is pressed
        break

print('Selected bounding boxes {}'.format(bboxes))
cv2.destroyAllWindows()
cap.release()

Press q to quit selecting boxes and start tracking
Press any other key to select next object
Press q to quit selecting boxes and start tracking
Press any other key to select next object
Press q to quit selecting boxes and start tracking
Press any other key to select next object
Press q to quit selecting boxes and start tracking
Press any other key to select next object
Press q to quit selecting boxes and start tracking
Press any other key to select next object
Press q to quit selecting boxes and start tracking
Press any other key to select next object
Press q to quit selecting boxes and start tracking
Press any other key to select next object
Press q to quit selecting boxes and start tracking
Press any other key to select next object
Press q to quit selecting boxes and start tracking
Press any other key to select next object
Press q to quit selecting boxes and start tracking
Press any other key to select next object
Selected bounding boxes [(331, 536, 442, 269), (949, 335, 409, 381), (

In [4]:

trackerType = "CSRT"   
 
# Create MultiTracker object
multiTracker = cv2.MultiTracker_create()
 
# Initialize MultiTracker
for bbox in bboxes:
    multiTracker.add(createTrackerByName(trackerType), frame, bbox)


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\core\src\dxt.cpp:3335: error: (-215:Assertion failed) type == CV_32FC1 || type == CV_32FC2 || type == CV_64FC1 || type == CV_64FC2 in function 'cv::dft'


In [5]:
import numpy as np
import cv2

cap = cv2.VideoCapture('surveillance.mp4')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

In [ ]:
while(1):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    ret, boxes = multiTracker.update(frame)
    for i, newbox in enumerate(boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(frame, p1, p2, colors[i], 2, 1)


    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()